# Análisis de la tasa de ocupación en vuelos nacionales dentro de EE.UU (1990-2009)

En este proyecto analizaremos la tasa de ocupación de vuelos (pasajeros/asientos) en los vuelos realizados dentro de los Estados Unidos en el período 1990-2009. 

Al ser una base de datos de grandes dimensiones (3606803 * 15) utilizaremos un contexto de pyspark para poder desplegar consultas de SQL en Big Data. 

La visualización de los datos la realizaremos en Tableau

## 1. Importación de librerías

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import findspark
findspark.init()
import pyspark
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

## 2. Inicialización de pyspark y contexto sql

In [2]:
sc = SparkSession.builder\
        .master("local")\
        .appName("Airports")\
        .config("spark.logLevel", "WARN")\
        .getOrCreate()

In [3]:
sqlContext = SQLContext(sc)

C:\Users\juanv\anaconda3\Lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
sc

## 3. Carga de datos

In [5]:
airports = sc.read.csv("Airports.csv", header=True, inferSchema=True)
airports.registerTempTable('airports')

C:\Users\juanv\anaconda3\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


## 4. Exploración / Limpieza de datos

In [6]:
print("Loaded dataset contains %d ruta * mes" % airports.count())
print("Loaded dataset contains %d columns" % len(airports.columns))

Loaded dataset contains 3606803 ruta * mes
Loaded dataset contains 15 columns


In [7]:
airports.printSchema()

root
 |-- Origin_airport: string (nullable = true)
 |-- Destination_airport: string (nullable = true)
 |-- Origin_city: string (nullable = true)
 |-- Destination_city: string (nullable = true)
 |-- Passengers: integer (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- Flights: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Fly_date: date (nullable = true)
 |-- Origin_population: integer (nullable = true)
 |-- Destination_population: integer (nullable = true)
 |-- Org_airport_lat: string (nullable = true)
 |-- Org_airport_long: string (nullable = true)
 |-- Dest_airport_lat: string (nullable = true)
 |-- Dest_airport_long: string (nullable = true)



In [79]:
airports.describe("Passengers", "Seats", "Flights", "Distance", "Origin_population", "Destination_population").show()

+-------+------------------+-----------------+------------------+-----------------+-----------------+----------------------+
|summary|        Passengers|            Seats|           Flights|         Distance|Origin_population|Destination_population|
+-------+------------------+-----------------+------------------+-----------------+-----------------+----------------------+
|  count|           3606803|          3606803|           3606803|          3606803|          3606803|               3606803|
|   mean|2688.9104331453646|4048.297368888736| 37.22889855642241|697.3190326724249| 5871502.49894491|      5897982.44118434|
| stddev|  4347.61704769634|6200.871210153885|49.619697799496414|604.4165108483492|7858061.601821028|     7906127.406405261|
|    min|                 0|                0|                 0|                0|            13005|                 12887|
|    max|             89597|           147062|              1128|             5095|         38139592|              38139592|


Las grandes desviaciones estándares de las variables Passengers, Seats y Flights y Distance nos da entender una gran variabilidad de los datos. Además, debemos tener en cuenta que el mínimo de dichas columnas es 0, lo que significa la posible existencia de vuelos sin pasajeros (probable), vuelos sin asientos (no probable), rutas sin vuelos, y rutas sin distancia (no probable). Por tal razón en la sección de limpieza de datos eliminaremos los registros que tengan 0 Seats y 0 Fligths. Los registros de 0 Distances lo manipularemos en Tableau.

In [9]:
airports.orderBy(airports["Passengers"].desc()).select("Origin_airport","Destination_airport","Passengers","Seats", "Flights", "Fly_date").show(15)

+--------------+-------------------+----------+------+-------+----------+
|Origin_airport|Destination_airport|Passengers| Seats|Flights|  Fly_date|
+--------------+-------------------+----------+------+-------+----------+
|           OGG|                HNL|     89597|116785|    960|2009-07-01|
|           OGG|                HNL|     87037|116326|    957|2009-08-01|
|           HNL|                OGG|     85759|112965|    930|2009-07-01|
|           SFO|                LAX|     83153|142128|   1128|1994-12-01|
|           OGG|                HNL|     82436|105421|    867|2009-09-01|
|           HNL|                OGG|     82347|112413|    926|2009-08-01|
|           LAX|                SFO|     82298|127764|   1014|1995-03-01|
|           OGG|                HNL|     82296|107929|    888|2009-03-01|
|           SFO|                LAX|     81322|127512|   1012|1995-03-01|
|           OGG|                HNL|     80548|101575|    840|1999-08-01|
|           LAX|                SFO|  

De manera preliminar podemos observar que los vuelos realizados entre OGG - NHL los meses de julio y agosto (verano) de 2009 son los que tienen más pasajeros. Podemos observar que en la lista de los vuelos con más pasajeros también se incluye la ruta contraria (HNL-OGG) y los vuelos que hacen la ruta LAX-SFO (ida y vuelta).

In [10]:
from pyspark.sql.functions import max, min
max_date = airports.agg(max("Fly_date")).collect()[0][0]
min_date = airports.agg(min("Fly_date")).collect()[0][0]
print("Fecha máxima de vuelo:", max_date)
print("Fecha mínima de vuelo:", min_date)

Fecha máxima de vuelo: 2009-12-01
Fecha mínima de vuelo: 1990-01-01


In [57]:
flights_zero = airports.filter((airports["Flights"] == 0) | (airports["Seats"] == 0))

# Mostrar el resultado
flights_zero.select("Origin_airport", "Destination_airport", "Passengers", "Seats", "Flights", "Fly_date").show()

+--------------+-------------------+----------+-----+-------+----------+
|Origin_airport|Destination_airport|Passengers|Seats|Flights|  Fly_date|
+--------------+-------------------+----------+-----+-------+----------+
|           SEA|                RDM|         0|    0|      0|1993-09-01|
|           SEA|                RDM|         0|    0|      0|1997-01-01|
|           PDX|                RDM|         0|    0|     25|2002-10-01|
|           PDX|                RDM|         0|    0|     27|2002-12-01|
|           PDX|                RDM|         0|    0|     26|2002-11-01|
|           PDX|                RDM|         0|    0|     25|2002-11-01|
|           PDX|                RDM|         0|    0|     25|2002-10-01|
|           PDX|                RDM|         0|    0|     27|2002-12-01|
|           PDX|                RDM|         0|    0|     23|2003-11-01|
|           PDX|                RDM|         0|    0|     24|2003-02-01|
|           PDX|                RDM|         0|    

Aquí podemos observar que hay registros que en pasajeros, sillas y vuelos tienen un 0. Tendríamos la opción de imputar estos valores por la mediana de cada ruta, pero para avanzar más rápido en el análisis borraremos estos registros.


In [82]:
flights_zero.count()

334186

Tengamos en cuenta que los registros que tienen 0 en vuelos o 0 en sillas representan un poco más del 9% del total de observaciones. Esto significa que no deberíamos borrarlos sino investigar por qué existen tantos datos que pueden ser considerados como posibles errores. En este ejercicio, optaremos por borrarlos por no contar con los recursos suficientes para indagar más al respecto. 

In [59]:
filtered_airports = airports.filter(~((airports["Flights"] == 0) | (airports["Seats"] == 0)))

In [60]:
filtered_airports.count()

3272617

## 5. SQL Queries

In [61]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

In [62]:
filtered_airports.createOrReplaceTempView("airports")

### 5.1 Pasajeros por aeropuerto de origen

In [63]:
query_1 = """
            SELECT Origin_airport, SUM(Passengers) AS Total_Passengers
            FROM airports
            GROUP BY Origin_airport
            ORDER BY Total_Passengers DESC
            LIMIT 10
          """
result_1 = sqlContext.sql(query_1)

# Mostrar los primeros 10 resultados
result_1.show(10)

+--------------+----------------+
|Origin_airport|Total_Passengers|
+--------------+----------------+
|           ATL|       577124268|
|           ORD|       529017907|
|           DFW|       457153720|
|           LAX|       393005635|
|           PHX|       295857703|
|           LAS|       270590248|
|           DTW|       250983023|
|           MSP|       245197238|
|           SFO|       243779917|
|           IAH|       228367406|
+--------------+----------------+



### 5.2 Pasajeros por aeropuerto de destino

In [64]:
query_2 = """
             SELECT Destination_airport, SUM(Passengers) AS Total_Passengers 
             FROM airports 
             GROUP BY Destination_airport 
             ORDER BY Total_Passengers DESC 
             LIMIT 10
          """

result_2 = sqlContext.sql(query_2)

result_2.show()

+-------------------+----------------+
|Destination_airport|Total_Passengers|
+-------------------+----------------+
|                ATL|       577954144|
|                ORD|       528648148|
|                DFW|       458322527|
|                LAX|       389476602|
|                PHX|       295580444|
|                LAS|       269144965|
|                DTW|       251467874|
|                MSP|       245774036|
|                SFO|       242283245|
|                IAH|       229104958|
+-------------------+----------------+



Los aeropuertos con mayor número de pasajeros saliendo y llegando son Atlanta (ATL) y Chicago (ORD)

### 5.3 Vuelos por aeropuerto de origen

In [65]:
query_3 = """
            SELECT Origin_airport, SUM(Flights) AS Total_Flights
            FROM airports
            GROUP BY Origin_airport
            ORDER BY Total_Flights DESC
            LIMIT 10
          """

result_3 = sqlContext.sql(query_3)

result_3.show()

+--------------+-------------+
|Origin_airport|Total_Flights|
+--------------+-------------+
|           ORD|      6818070|
|           ATL|      6503131|
|           DFW|      5935095|
|           LAX|      3980655|
|           DTW|      3435617|
|           MSP|      3177751|
|           IAH|      3173117|
|           PHX|      3169851|
|           STL|      3167024|
|           CLT|      2825912|
+--------------+-------------+



### 5.4 Vuelos por aeropuerto de destino

In [66]:
query_4 = """
            SELECT Destination_airport, SUM(Flights) AS Total_Flights
            FROM airports
            GROUP BY Destination_airport
            ORDER BY Total_Flights DESC
            LIMIT 10
          """

result_4 = sqlContext.sql(query_4)

result_4.show()

+-------------------+-------------+
|Destination_airport|Total_Flights|
+-------------------+-------------+
|                ORD|      6803504|
|                ATL|      6492397|
|                DFW|      5924464|
|                LAX|      3979424|
|                DTW|      3431092|
|                MSP|      3173187|
|                IAH|      3171937|
|                PHX|      3164813|
|                STL|      3162735|
|                CLT|      2821610|
+-------------------+-------------+



Los aeropuertos de donde más salen y llegan vuelos son Chicago (ORD) y Atlanta (ATL)

### 5.5 Pasajeros por mes del vuelo 

In [67]:
query_5 = """
             SELECT Fly_date, sum(Passengers) AS Passengers 
             FROM airports 
             GROUP BY Fly_date
             ORDER BY Passengers DESC 
             LIMIT 10
          """

result_5 = sqlContext.sql(query_5)

result_5.show()

+----------+----------+
|  Fly_date|Passengers|
+----------+----------+
|2007-07-01|  54064801|
|2005-07-01|  53860117|
|2007-08-01|  53572119|
|2007-06-01|  52602824|
|2008-07-01|  52343636|
|2006-07-01|  52067303|
|2005-06-01|  51732595|
|2004-07-01|  51671090|
|2005-08-01|  51223855|
|2006-06-01|  50900094|
+----------+----------+



El mayor número de pasajeros se registró en los meses de julio de 2005 y 2007

### 5.6 Vuelos por mes del vuelo 

In [68]:
query_6 = """
             SELECT Fly_date, sum(Flights) AS Flights 
             FROM airports 
             GROUP BY Fly_date
             ORDER BY Flights DESC 
             LIMIT 10
          """

result_6 = sqlContext.sql(query_6)

result_6.show()

+----------+-------+
|  Fly_date|Flights|
+----------+-------+
|2005-08-01| 706107|
|2004-08-01| 702789|
|2005-07-01| 700542|
|2005-05-01| 699656|
|2004-10-01| 697235|
|2005-03-01| 695812|
|2004-07-01| 688060|
|2005-06-01| 685739|
|2007-08-01| 678718|
|2005-04-01| 676282|
+----------+-------+



El mayor número de vuelos se registró en los meses de agosto de 2004 y 2005. Observemos que las fechas de mayor número de pasajeros y mayor número de vuelos no coinciden

### 5.7 Vuelos totales (salidas y llegadas) por aeropuerto 

In [69]:
query_7 = """
             WITH 
                 destination AS (
                     SELECT Destination_airport as Airport, SUM(Flights) AS Out_Flights 
                     FROM airports 
                     GROUP BY Destination_airport),
                 origin AS (
                     SELECT Origin_airport as Airport, SUM(Flights) AS In_Flights 
                     FROM airports 
                     GROUP BY Origin_airport)
                 SELECT origin.Airport, (destination.Out_Flights+origin.In_Flights) AS Total_Flights
                 FROM origin, destination 
                 WHERE origin.Airport = destination.Airport
                 ORDER BY (Total_Flights) DESC
                 LIMIT 15;
          """

result_7 = sqlContext.sql(query_7)

result_7.show()

+-------+-------------+
|Airport|Total_Flights|
+-------+-------------+
|    ORD|     13621574|
|    ATL|     12995528|
|    DFW|     11859559|
|    LAX|      7960079|
|    DTW|      6866709|
|    MSP|      6350938|
|    IAH|      6345054|
|    PHX|      6334664|
|    STL|      6329759|
|    CLT|      5647522|
|    EWR|      5479413|
|    LGA|      5368708|
|    PHL|      5225478|
|    SEA|      5067077|
|    LAS|      5008002|
+-------+-------------+



### 5.8 Pasajeros totales (salidas y llegadas) por aeropuerto 

In [70]:
query_8 = """
             WITH 
                 destination AS (
                     SELECT Destination_airport as Airport, SUM(Passengers) AS Out_Passengers 
                     FROM airports 
                     GROUP BY Destination_airport),
                 origin AS (
                     SELECT Origin_airport as Airport, SUM(Passengers) AS In_Passengers 
                     FROM airports 
                     GROUP BY Origin_airport)
                 SELECT origin.Airport, (destination.Out_Passengers+origin.In_Passengers) AS Total_Passengers
                 FROM origin, destination 
                 WHERE origin.Airport = destination.Airport
                 ORDER BY (Total_Passengers) DESC
                 LIMIT 15;
          """

result_8 = sqlContext.sql(query_8)

result_8.show()

+-------+----------------+
|Airport|Total_Passengers|
+-------+----------------+
|    ATL|      1155078412|
|    ORD|      1057666055|
|    DFW|       915476247|
|    LAX|       782482237|
|    PHX|       591438147|
|    LAS|       539735213|
|    DTW|       502450897|
|    MSP|       490971274|
|    SFO|       486063162|
|    IAH|       457472364|
|    MCO|       453295762|
|    SEA|       424279043|
|    EWR|       418210330|
|    STL|       409513281|
|    CLT|       406475433|
+-------+----------------+



Los aeropueros de Atlanta (Atl) y Chicago (ORD) son lo que mayor número de vuelos y pasajeros tienen en consolidados de vuelos y pasajeros (llegadas y salidas)

### 5.9 Cálculo de la tasa de ocupación (pasajeros/asientos)

In [91]:
query_16 = """
            SELECT SUM(Passengers) AS Total_Passengers,
                   SUM(Seats) AS Total_Seats,
                   ROUND((SUM(Passengers)*100/SUM(Seats)), 2) AS Occupancy_Rate  
            FROM airports
"""

result_16 = sqlContext.sql(query_16)

result_16.show()

+----------------+-----------+--------------+
|Total_Passengers|Total_Seats|Occupancy_Rate|
+----------------+-----------+--------------+
|      9698351442|14601328755|         66.42|
+----------------+-----------+--------------+



La tasa de ocupación de todas las rutas durante todo el período fue de 66.42

### 5.10 Min de Tasa de ocupación por ruta (ida y vuelta)

In [72]:
query_9 = """

            WITH table1 AS 
                (
                SELECT LEAST(Origin_airport, Destination_airport) AS Airport1, 
                       GREATEST(Destination_airport, Origin_airport) AS Airport2, 
                       SUM(Flights) AS Flights,
                       SUM(Passengers) AS Passengers,
                       SUM(Seats) AS Seats
                FROM airports
                GROUP BY Airport1, Airport2
                ORDER BY 1,2
                )
            SELECT t.*, ROUND((Passengers*100/Seats), 4) AS Occupancy_Rate
            FROM table1 t 
            WHERE (Passengers * 100 / Seats) IS NOT NULL
            AND (Passengers * 100 / Seats) > 0
            AND (Passengers * 100 / Seats) <= 100
            ORDER BY Occupancy_Rate 
            LIMIT 15;
          """

result_9 = sqlContext.sql(query_9)

result_9.show()

+--------+--------+-------+----------+-----+--------------+
|Airport1|Airport2|Flights|Passengers|Seats|Occupancy_Rate|
+--------+--------+-------+----------+-----+--------------+
|     AFW|     JFK|     14|         2| 3459|        0.0578|
|     LCK|     TUL|     32|         5| 6838|        0.0731|
|     ACY|     NZC|      9|         1| 1098|        0.0911|
|     ACT|     JFK|      5|         1|  983|        0.1017|
|     MEM|     RFD|      7|         1|  899|        0.1112|
|     SYR|     YNG|      7|         1|  852|        0.1174|
|     MIA|     TMB|     51|         8| 5915|        0.1352|
|     LAF|     MIA|      9|         2| 1397|        0.1432|
|     MIA|     POB|     12|         4| 2235|         0.179|
|     LBB|     PUB|      4|         1|  452|        0.2212|
|     AEX|     BOS|     36|        18| 7584|        0.2373|
|     BHM|     ILM|      3|         1|  329|         0.304|
|     GSP|     RFD|      2|         1|  321|        0.3115|
|     FAT|     HLN|      4|         2|  

Descubrimos que la ruta con menos tasa de ocupación es Fort Worth, TX (AFW) - Nueva York (JFK). Al ser tan baja (0.05%) inspeccionamos con mayor detalle esta ruta

### 5.11 Detalle ruta AFW - JFK (ida y vuelta)

In [73]:
query_15 = """
            SELECT Origin_airport, Destination_airport, 
               SUM(Flights) AS Total_Flights, 
               SUM(Passengers) AS Total_Passengers,
               SUM(Seats) AS Total_Seats,
               ROUND(MEAN(Distance), '0.00') AS Distance
            FROM airports
            WHERE Origin_airport IN ('AFW', 'JFK') 
            AND Destination_airport IN ('AFW', 'JFK')
            GROUP BY Origin_airport, Destination_airport

"""
result_15 = sqlContext.sql(query_15)

result_15.show()

+--------------+-------------------+-------------+----------------+-----------+--------+
|Origin_airport|Destination_airport|Total_Flights|Total_Passengers|Total_Seats|Distance|
+--------------+-------------------+-------------+----------------+-----------+--------+
|           JFK|                AFW|            8|               2|       2002|  1402.0|
|           JFK|                JFK|           52|            4846|       8875|     0.0|
|           AFW|                JFK|            6|               0|       1457|  1402.0|
+--------------+-------------------+-------------+----------------+-----------+--------+



Hallamos que en la base de datos existen registros de vuelos cuyo aeropuerto de salida y de entrada son el mismo, en este caso (JFK -JFK). Esta observación también tendría que ser investigada con mayor detenimiento. Por ahora, lo que haremos será filtrar la base de datos en Tableau eliminado los registros cuya distancia es 0

### 5.12 Max de Tasa de ocupación por ruta (ida y vuelta) y número de vuelos

In [74]:
query_10 = """

            WITH table1 AS 
                (
                SELECT LEAST(Origin_airport, Destination_airport) AS Airport1, 
                       GREATEST(Destination_airport, Origin_airport) AS Airport2, 
                       SUM(Flights) AS Flights,
                       SUM(Passengers) AS Passengers,
                       SUM(Seats) AS Seats
                FROM airports
                GROUP BY Airport1, Airport2
                ORDER BY 1,2
                )
            SELECT t.*, (Passengers*100/Seats) AS Occupancy_Rate
            FROM table1 t 
            WHERE (Passengers * 100 / Seats) IS NOT NULL
            AND (Passengers * 100 / Seats) > 0
            AND (Passengers * 100 / Seats) <= 100
            ORDER BY Occupancy_Rate DESC, Flights DESC
            LIMIT 15;
          """

result_10 = sqlContext.sql(query_10)

result_10.show()

+--------+--------+-------+----------+-----+--------------+
|Airport1|Airport2|Flights|Passengers|Seats|Occupancy_Rate|
+--------+--------+-------+----------+-----+--------------+
|     IAD|     MCW|      6|       972|  972|         100.0|
|     BIS|     IAD|      4|       648|  648|         100.0|
|     IND|     OSH|      4|       650|  650|         100.0|
|     DQF|     SLN|      3|       548|  548|         100.0|
|     FTW|     PVU|      2|       300|  300|         100.0|
|     AMA|     MOD|      2|       100|  100|         100.0|
|     ALO|     MSO|      2|       340|  340|         100.0|
|     CSG|     LUK|      2|       100|  100|         100.0|
|     PDX|     TIW|      2|        18|   18|         100.0|
|     ASL|     MEM|      2|        16|   16|         100.0|
|     BZN|     OKC|      2|       100|  100|         100.0|
|     ALO|     GGG|      2|       168|  168|         100.0|
|     CLL|     LWC|      2|        60|   60|         100.0|
|     LWC|     OKC|      2|        38|  

Las 15 rutas que tienen la mayor capacidad de ocupación (100%) presentan muy pocos vuelos en todo el período (entre 6 y 2 durante 20 años)

### 5.13 Max de Tasa de ocupación por ruta (ida y vuelta), vuelos, asientos y pasajeros

In [75]:
query_11 = """

            WITH table1 AS 
                (
                SELECT LEAST(Origin_airport, Destination_airport) AS Airport1, 
                       GREATEST(Destination_airport, Origin_airport) AS Airport2, 
                       SUM(Flights) AS Flights,
                       SUM(Passengers) AS Passengers,
                       SUM(Seats) AS Seats
                FROM airports
                GROUP BY Airport1, Airport2
                ORDER BY 1,2
                )
            SELECT t.*, ROUND((Passengers*100/Seats), '0.00') AS Occupancy_Rate
            FROM table1 t 
            WHERE (Passengers * 100 / Seats) IS NOT NULL
            AND (Passengers * 100 / Seats) > 0
            AND (Passengers * 100 / Seats) <= 100
            ORDER BY Flights DESC, Seats DESC, Passengers DESC, Occupancy_Rate DESC
            LIMIT 15;
          """

result_11 = sqlContext.sql(query_11)

result_11.show()

+--------+--------+-------+----------+--------+--------------+
|Airport1|Airport2|Flights|Passengers|   Seats|Occupancy_Rate|
+--------+--------+-------+----------+--------+--------------+
|     HNL|     OGG| 755662|  62109354|96640901|          64.0|
|     LAX|     SFO| 600392|  51119989|79405656|          64.0|
|     LAS|     LAX| 586580|  52511530|80532768|          65.0|
|     PDX|     SEA| 560063|  18475771|34650955|          53.0|
|     LAX|     PHX| 514206|  42695385|65619395|          65.0|
|     BOS|     LGA| 470736|  31242486|64897330|          48.0|
|     MSP|     ORD| 465125|  31301666|55325318|          57.0|
|     LAS|     PHX| 455608|  42979048|64844100|          66.0|
|     DCA|     LGA| 439107|  29471657|60663368|          49.0|
|     LAX|     SAN| 430584|  11686171|22820096|          51.0|
|     LGA|     ORD| 424269|  39981416|59616532|          67.0|
|     DAL|     HOU| 408272|  35573141|53054549|          67.0|
|     ATL|     DFW| 390078|  42941213|59978776|        

Relacionando la tasa de ocupación con el mayor número de vuelos, pasajeros y sillas podemos determinar que las dos rutas que mejor cumplieron la meta fueron las cubiertas entre los aeropuertos de Honolulu (NHL) y Kahului (OGG) y Los Ángeles (LAX) y San Francisco (SFO). Recordemos que ya habíamos determinado que estas rutas tuvieron la mayor cantidad de pasajeros en el período 

### 5.14 Número de vuelos por distancia

In [76]:
query_12 = """
             WITH table1 AS 
                (
                SELECT LEAST(Origin_airport, Destination_airport) AS Airport1, 
                       GREATEST(Destination_airport, Origin_airport) AS Airport2, 
                       ROUND(MEAN(Distance), '0.00') AS Distance,
                       SUM(Flights) AS Flights
                FROM airports
                GROUP BY Airport1, Airport2
                ORDER BY 1,2
                )
            SELECT t.*
            FROM table1 t
            WHERE Flights > 0
            ORDER BY Flights DESC
            LIMIT 15;
          """

result_12 = sqlContext.sql(query_12)

result_12.show()

+--------+--------+--------+-------+
|Airport1|Airport2|Distance|Flights|
+--------+--------+--------+-------+
|     HNL|     OGG|   100.0| 755662|
|     LAX|     SFO|   337.0| 600392|
|     LAS|     LAX|   236.0| 586580|
|     PDX|     SEA|   129.0| 560063|
|     LAX|     PHX|   370.0| 514206|
|     BOS|     LGA|   185.0| 470736|
|     MSP|     ORD|   334.0| 465125|
|     LAS|     PHX|   256.0| 455608|
|     DCA|     LGA|   214.0| 439107|
|     LAX|     SAN|   109.0| 430584|
|     LGA|     ORD|   733.0| 424269|
|     DAL|     HOU|   239.0| 408272|
|     ATL|     DFW|   732.0| 390078|
|     LAX|     OAK|   337.0| 371652|
|     EWR|     ORD|   719.0| 368692|
+--------+--------+--------+-------+



Revisamos la distancia de rutas con mayor cantidad de vuelos: Honolulu (NHL) y Kahului (OGG) y Los Ángeles (LAX) y San Francisco (SFO)

### 5.15 Max de distancia por número de vuelos

In [77]:
query_13 = """
             WITH table1 AS 
                (
                SELECT LEAST(Origin_airport, Destination_airport) AS Airport1, 
                       GREATEST(Destination_airport, Origin_airport) AS Airport2, 
                       ROUND(MEAN(Distance), '0.00') AS Distance,
                       SUM(Flights) AS Flights
                FROM airports
                GROUP BY Airport1, Airport2
                ORDER BY 1,2
                )
            SELECT t.*
            FROM table1 t
            ORDER by Distance DESC
            limit 15;
          """

result_13 = sqlContext.sql(query_13)

result_13.show()

+--------+--------+--------+-------+
|Airport1|Airport2|Distance|Flights|
+--------+--------+--------+-------+
|     BDL|     HNL|  5018.0|      1|
|     HNL|     JFK|  4983.0|     80|
|     HIK|     JFK|  4983.0|      2|
|     HNL|     LGA|  4976.0|      1|
|     EWR|     HNL|  4962.0|   8318|
|     JFK|     OGG|  4924.0|      1|
|     HNL|     PHL|  4919.0|      1|
|     HNL|     ISO|  4860.0|      1|
|     OGG|     PHL|  4859.0|      1|
|     HNL|     IAD|  4817.0|      3|
|     BWI|     OGG|  4793.0|      1|
|     OGG|     RIC|  4780.0|      1|
|     HNL|     SFB|  4755.0|      2|
|     OGG|     RDU|  4722.0|      1|
|     MCO|     OGG|  4679.0|      1|
+--------+--------+--------+-------+



La mayor cantidad de rutas de larga distancia presentan muy pocos registros de vuelos durante el período. Sin embargo, la ruta Newark (EWR) - Honolulu (NHL) (ida y vuelta) tiene 8318 en el período. Inspeccionaremos más a fondo esta conclusión en Tableau

### 5.16 Ocupación por número de vuelos y fecha 


In [94]:
query_14 = """
             WITH table1 AS 
                (
                SELECT LEAST(Origin_airport, Destination_airport) AS Airport1, 
                       GREATEST(Destination_airport, Origin_airport) AS Airport2, 
                       Fly_Date AS Date,
                       SUM(Flights) AS Flights,
                       SUM(Passengers) AS Passengers,
                       SUM(Seats) AS Seats
                FROM airports
                GROUP BY Airport1, Airport2, Date
                ORDER BY 1,2
                )
            SELECT t.*, ROUND((Passengers*100/Seats), '0.00') AS Occupancy_Rate
            FROM table1 t
            ORDER by Flights DESC, Date 
            limit 15;
          """

result_14 = sqlContext.sql(query_14)

result_14.show()

+--------+--------+----------+-------+----------+------+--------------+
|Airport1|Airport2|      Date|Flights|Passengers| Seats|Occupancy_Rate|
+--------+--------+----------+-------+----------+------+--------------+
|     LAX|     SFO|1990-08-01|   5103|    401424|637255|          63.0|
|     LAX|     SFO|1990-10-01|   5059|    321963|626031|          51.0|
|     LAX|     SFO|1990-07-01|   5053|    307586|621239|          50.0|
|     LAX|     SFO|1990-09-01|   4878|    314631|599524|          52.0|
|     LAX|     SFO|1990-06-01|   4854|    281987|602955|          47.0|
|     LAX|     SFO|1990-12-01|   4655|    343118|602276|          57.0|
|     LAX|     SFO|1990-11-01|   4597|    348875|612257|          57.0|
|     HNL|     OGG|1995-08-01|   4442|    393851|640874|          61.0|
|     HNL|     OGG|1994-08-01|   4407|    391786|632637|          62.0|
|     HNL|     OGG|1995-07-01|   4397|    373172|637710|          59.0|
|     LAX|     SFO|1991-01-01|   4257|    340045|546794|        

Las rutas con mayor tasa de ocupación (entre 47% y 63%) y mayor número de vuelos pertenecen a las rutas (ida y vuelta) entre Los Ángeles (LAX) y San Francisco (SFO) y Honolulu (NHL) y Kahului (OGG) en diferentes fechas. Los meses con mayor número de vuelos y mejor tasa de ocupación fueron agosto y octubre de 1990 de la ruta Los Ángeles (LAX) - San Francisco (SFO)